In [199]:
import pandas as pd
from sklearn.utils import shuffle
from scipy.io import arff
from sklearn import datasets
from sklearn import neighbors
from sklearn import metrics
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from tabulate import tabulate
import numpy as np

import time
import datetime

In [15]:
bone_marrow_transplant_children = fetch_ucirepo(id=565)

In [4]:
diabetesRepo = fetch_ucirepo(id=296)

/opt/homebrew/lib/python3.11/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


# Prepare datasets

In [196]:
randomState = 22020
labelEncoder = LabelEncoder()
boneData, boneTarget = shuffle(bone_marrow_transplant_children.data.features, bone_marrow_transplant_children.data.targets, random_state=randomState)
diabetesData, diabetesTarget = shuffle(diabetesRepo.data.features, diabetesRepo.data.targets, random_state=randomState)

numericBoneData = boneData.select_dtypes(include='number')
numericBoneTarget = boneTarget.select_dtypes(include='number')

numericDiabetesData = diabetesData.select_dtypes(include='number')
numericDiabetesTarget = labelEncoder.fit_transform(diabetesTarget)

display(numericDiabetesTarget)

numericBoneData = numericBoneData.fillna(0)
numericBoneTarget = numericBoneTarget.fillna(0)

xBoneTrain, xBoneTest, yBoneTrain, yBoneTest = train_test_split(numericBoneData, numericBoneTarget, test_size=0.33, random_state=(randomState+1))
xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest = train_test_split(numericDiabetesData, numericDiabetesTarget, test_size=0.33, random_state=(randomState+1))

/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([2, 0, 1, ..., 2, 2, 2])

## Helper functions

In [160]:
columns = ["Model", "Accuracy", "Precision", "Training Time", "Testing Time"]
def printTable(cols, data):
    print(tabulate(data, headers=cols, tablefmt="grid"))

def getMeasures(classifier, X_train, X_test, y_train, y_test):
    startTimeTrain = time.time()
    classifier.fit(X_train, y_train)
    endTimeTrain = time.time()
    trainingTime = endTimeTrain - startTimeTrain

    startTimeTest = time.time()
    yTestPredicted = classifier.predict(X_test)
    endTimeTest = time.time()
    testingTime = endTimeTest - startTimeTest

    accuracy = metrics.accuracy_score(y_test, yTestPredicted)
    precision = metrics.precision_score(y_test, yTestPredicted, average="micro")
    return (accuracy, precision, trainingTime, testingTime)

## KNN

In [145]:
noNeighbors = [10, 20, 40]

def calcKNN(X_train, X_test, y_train, y_test):
    data = []
    for neighbor in noNeighbors:
        classifier = neighbors.KNeighborsClassifier(neighbor)
        
        accuracy, precision, trainingTime, testingTime = getMeasures(classifier, X_train, X_test, y_train, y_test)
        dataIteration = ["KNN (" + str(neighbor) + " Neighbors)", str(accuracy), str(precision), str(trainingTime) + "s", str(testingTime) + "s"]
        data.append(dataIteration)
    return data


## Perceptron

In [146]:
# With the help of https://www.geeksforgeeks.org/sklearn-perceptron/, but barely.

pLearningRate = 0.1
iterations = 50

def calcPerceptron(X_train, X_test, y_train, y_test):
    data = []

    classifier = Perceptron(max_iter=iterations, eta0=0.1, random_state=randomState)
    accuracy, precision, trainingTime, testingTime = getMeasures(classifier, X_train, X_test, y_train, y_test)
    dataIteration = ["Perceptron", str(accuracy), str(precision), str(trainingTime) + "s", str(testingTime) + "s"]
    data.append(dataIteration)
    
    return data



## Naive Bayes (Gaussian)

In [169]:

def calcGaussianNaiveBayes(X_train, X_test, y_train, y_test):
    data = []

    classifier = GaussianNB()
    accuracy, precision, trainingTime, testingTime = getMeasures(classifier, X_train, X_test, y_train, y_test)
    dataIteration = ["Naive Bayes (Gaussian)", str(accuracy), str(precision), str(trainingTime) + "s", str(testingTime) + "s"]
    data.append(dataIteration)

    return data

## Naive Bayes (Bernoulli)

In [170]:
def calcBernoulliNaiveBayes(X_train, X_test, y_train, y_test):
    data = []

    classifier = BernoulliNB()
    accuracy, precision, trainingTime, testingTime = getMeasures(classifier, X_train, X_test, y_train, y_test)
    dataIteration = ["Naive Bayes (Bernoulli)", str(accuracy), str(precision), str(trainingTime) + "s", str(testingTime) + "s"]
    data.append(dataIteration)

    return data

In [200]:
def calcMultinominalNaiveBayes(X_train, X_test, y_train, y_test):
    data = []

    classifier = MultinomialNB()
    accuracy, precision, trainingTime, testingTime = getMeasures(classifier, X_train, X_test, y_train, y_test)
    dataIteration = ["Naive Bayes (Multinominal)", str(accuracy), str(precision), str(trainingTime) + "s", str(testingTime) + "s"]
    data.append(dataIteration)

    return data

## Bone Marrow dataset

In [201]:
boneMarrowTable = []

boneMarrowTable += calcKNN(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcPerceptron(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcGaussianNaiveBayes(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcBernoulliNaiveBayes(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcMultinominalNaiveBayes(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)

printTable(columns, boneMarrowTable)

/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ),

ValueError: Negative values in data passed to MultinomialNB (input X)

## Diabetes dataset

In [172]:
diabetesTable = []

diabetesTable += calcKNN(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)
diabetesTable += calcPerceptron(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)
diabetesTable += calcGaussianNaiveBayes(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)
diabetesTable += calcBernoulliNaiveBayes(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)

printTable(columns, diabetesTable)

+-------------------------+------------+-------------+-----------------------+------------------------+
| Model                   |   Accuracy |   Precision | Training Time         | Testing Time           |
+=========================+============+=============+=======================+========================+
| KNN (10 Neighbors)      |   0.530685 |    0.530685 | 0.03760480880737305s  | 1.9086682796478271s    |
+-------------------------+------------+-------------+-----------------------+------------------------+
| KNN (20 Neighbors)      |   0.550755 |    0.550755 | 0.02330803871154785s  | 2.117655038833618s     |
+-------------------------+------------+-------------+-----------------------+------------------------+
| KNN (40 Neighbors)      |   0.558824 |    0.558824 | 0.02327585220336914s  | 2.559847116470337s     |
+-------------------------+------------+-------------+-----------------------+------------------------+
| Perceptron              |   0.548164 |    0.548164 | 0.0719208